$\Large\color{blue}{\text{PyGMTSAR Yamchi DAM Interferograms Timeseries Analysis}}$

### See and more PyGMTSAR notebooks on GitHub: [PyGMTSAR](https://github.com/mobigroup/gmtsar)

It works on

* MacOS Monterey (Apple Silicon) and BigSur (Intel) (Python 3.9) - please pre-install system dependencies (maybe using HomeBrew),

* Google Cloud VM and Notebooks on Debian 10 (Python 3.7), use the [Google Cloud Debian 10 init script](https://github.com/mobigroup/gmtsar/blob/master/gmtsar/sh/GMTSAR.install.debian10.sh) and [Google Cloud Debian 10 VM creation script](https://github.com/mobigroup/gmtsar/blob/master/gmtsar/sh/GMTSAR.gcloud_create_debian10.sh)

* $\color{red}{\text{Google Colab (Python 3.7) - you will be asked to re-run the notebook once due to "system crash" by desing}}$
$\color{red}{\text{Note: to open all notebook cells select menu "View" -> "Expand Sections"}}$

### PyGMTSAR is my free-time Open Source project.

That's a bit curious how the project was started a year ago. I develop geophysical inversion methods and processing software for many years using my fundamental physics and mathematics background. Satellite interferometry is the key point to validate my inversion models and I found the same problem as you too that the existing interferometry packages usage is a pain. There is no interactive processing with multiprocessing and even a progressbar and ability to view and change every step code and validate the results. Also, many used algorithms are too outdated and produce terrible results like to tension surfaces in GMT which is used widely in GMTSAR (hmm, how about to control a smothness of derivative? Tension surfaces were invented when all the Earths computers where less powerfull than your smarthone today. If you are interested I shared the examples in GMTSAR bug tracker). Anyway, I found GMT mathematics is really crazy and the developers replace one incorrect algorithm by another and back often as we see in the codes and the changelog). That was enough reason to check all the used algorithms and replace these by modern and correct ones. By this way, I use only GMTSAR C codes with my patches to fix some errors and allow interoperability with Python wrappers plus my own codes around them. GMTSAR codes are fine and without crazy GMT codes work better and the processing is much faster. Alright, I spent one month to make the initial PyGMTSAR realization and it works. Recently, I returned to the project to add some more sophisticated features like to scenes and subswathes stitching. I'm going to share some of my geological exprorations and seismic models as live examples on Google Colab as soon as it will be possible to do. How lineaments and ore zones are related to interferograms? How gas and oil deposits are related to surface movements on interferometry displacement maps? I have the answer and I work on the tools to model and visualize them.

You'd find my theoretical models and processing codes foir geophisical inversions in Github repository https://github.com/mobigroup/gis-snippets and tools for the 4D results vizualization in https://github.com/mobigroup/ParaView-plugins

Ah yes, a little bit about me. I have STEM master's degree in radio physics and in 2004 I was awarded first prize of the All-Russian Physics competition for significant results in Inverse modeling for non-linear optics and holography, also applicable for Inverse Modeling of Gravity, Magnetic, and Thermal fields. In addition to my fundamental science knowledge, I’m world class data scientist and software developer with 20 years experience in science and industrial development. I have worked on government contracts and universities projects and on projects for LG Corp, Google Inc, etc. You are able to find some of my software and results on LinkedIn and GitHub and Upwork. By the way, I left Russia many years ago and I work remotely for about 20 years.

### To order some research, development and support see my profile on freelance platform [Upwork](https://www.upwork.com/freelancers/~01e65e8e7221758623)

### @ Alexey Pechnikov, August, 2022

[Geological models on YouTube channel](https://www.youtube.com/channel/UCSEeXKAn9f_bDiTjT6l87Lg)

[Augmented Reality (AR) Geological Models](https://mobigroup.github.io/ParaView-Blender-AR/)

[GitHub repositories](https://github.com/mobigroup)

[English posts and articles on LinkedIn](https://www.linkedin.com/in/alexey-pechnikov/)

[Russian articles on Habr](https://habr.com/ru/users/N-Cube/posts/)

$\large\color{blue}{\text{Hint: Use menu Cell} \to \text{Run All or Runtime} \to \text{Complete All or Runtime} \to \text{Run All}}$
$\large\color{blue}{\text{(depending of your localization settings) to execute the entire notebook}}$

## Load Modules to Check Environment

In [ ]:
import platform, sys, os

## Debian 10 and Google Colab GMTSAR Installation

### On Google Cloud AI Notebooks: check root access

On Google Cloud AI Notebooks sometimes we have an issue when "sudo" requires a password. In this case drop the instance and create a new one and - that's important - wait 5-10 minutes before connect to it using link "OPEN JUPYTERLAB"

In [ ]:
if platform.system() == 'Linux':
    !sudo date

### Install https://github.com/mobigroup/gmtsar

I make lots of changes on GMTSAR C-coded tools and some of them are not merged to the upstream GMTSAR yet because all the patches need to be validated and discussed before. Also, my Python extensions are provided in my GMTSAR fork only. I hope in the future to provide a standalone python packager as wrapper around upstream GMTSAR but there is a long way to it.

In [ ]:
if platform.system() == 'Linux':
    count = !ls /usr/local | grep GMTSAR | wc -l
    if count == ['0']:
        !apt install -y csh autoconf gfortran \
            libtiff5-dev libhdf5-dev liblapack-dev libgmt-dev gmt-dcw gmt-gshhg gmt > /dev/null
        !cd /usr/local && git clone --branch master https://github.com/mobigroup/gmtsar GMTSAR > /dev/null
        !cd /usr/local/GMTSAR && autoconf > /dev/null
        !cd /usr/local/GMTSAR && ./configure --with-orbits-dir=/tmp > /dev/null
        !cd /usr/local/GMTSAR && make 1>/dev/null 2>/dev/null
        !cd /usr/local/GMTSAR && make install >/dev/null

## Define ENV Variables for Jupyter Instance

In [ ]:
# use default GMTSAR installation path
GMTSAR = '/usr/local/GMTSAR'
PATH = os.environ['PATH']

if PATH.find('GMTSAR') == -1:
    PATH = os.environ['PATH'] + f':{GMTSAR}/bin/'
    %env PATH {PATH}
    %env GMTSAR {GMTSAR}

## Install Python Modules

```Maybe you need to restart your notebook, follow the instructions printing below```

```The installation takes a long time on fresh Debian 10 and a short time on Google Colab```

In [ ]:
!{sys.executable} --version

In [ ]:
if platform.system() == 'Linux':
    !{sys.executable} -m pip install cartopy==0.19.0.post1 1>/dev/null 2>/dev/null
    !{sys.executable} -m pip install xarray==0.19.0        1>/dev/null 2>/dev/null
    !{sys.executable} -m pip install scipy==1.7.1          1>/dev/null 2>/dev/null

In [ ]:
if platform.system() == 'Linux':
    !{sys.executable} -m pip install \
        h5py netcdf4 h5netcdf \
        rasterio rioxarray numpy \
        scikit-image scipy sklearn \
        xarray dask distributed zarr nc-time-axis \
        pandas geopandas \
        sentineleof elevation \
        matplotlib seaborn geoviews hvplot datashader bokeh \
        xmltodict joblib tqdm 1>/dev/null 2>/dev/null

$\large\color{red}{\text{Attention: On Google Colab we need to restart kernel once when modules installed}}$

$\large\color{blue}{\text{Hint: Use menu Cell} \to \text{Run All or Runtime} \to \text{Complete All or Runtime} \to \text{Run All}}$
$\large\color{blue}{\text{(depending of your localization settings) to execute the entire notebook}}$

In [ ]:
if platform.system() == 'Linux':
    import xarray
    import time
    print (xarray.__version__)
    if xarray.__version__ != '0.19.0':
        print ("""
    ***********************************************************************************
    *
    Do not worry, runtime is stopped by design. Please run the notebook again.
    *
    ***********************************************************************************
    """)
        time.sleep(1)
        os.kill(os.getpid(), 9)

## Load and Setup Python Modules

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
# supress numpy warnings
import warnings
warnings.filterwarnings('ignore')
import datetime

In [ ]:
# plotting modules
import hvplot.pandas  # noqa
import hvplot.xarray  # noqa
import holoviews as hv
pd.options.plotting.backend = 'hvplot'
from IPython.display import Image
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

In [ ]:
# define Pandas display settings
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

gstiles = hv.Tiles('https://mt1.google.com/vt/lyrs=s&x={X}&y={Y}&z={Z}', name='Google Satellite')
ottiles = hv.Tiles('https://tile.opentopomap.org/{Z}/{X}/{Y}.png', name='Open Topo')

## Load Custom Python Modules

In [ ]:
sys.path.append(os.path.join(os.environ['GMTSAR'],'gmtsar', 'py'))

from PRM import PRM
from SBAS import SBAS

## Download and unpack Sentinel-1 scenes

$\large\color{blue}{\text{Downloading cropped scenes from iCloud archive file}}$

In [ ]:
icloud_urls = {'yamchi_2021.tar.gz': 'https://www.icloud.com/iclouddrive/08eH4fzPmD87xTa_jSTIviAhA#yamchi_2021'}

if platform.system() == 'Linux':
  !apt install -y jq > /dev/null
import os
for fname in icloud_urls.keys():
  if os.path.isfile(fname):
    print ('Already exists iCloud file', fname)
    break
  print ('Downloading iCloud file', fname)
  uid = icloud_urls[fname].split('/')[-1].split('#')[0]
  request = f'{{"shortGUIDs":[{{"value":"{uid}"}}]}}'
  urls = !curl -s 'https://ckdatabasews.icloud.com/database/1/com.apple.cloudkit/production/public/records/resolve' \
    --data-raw '{request}' --compressed | jq -r '.results[0].rootRecord.fields.fileContent.value.downloadURL'
  !curl -so '{fname}' '{urls[0]}'

In [ ]:
# unpack the archive
!tar -zxvf yamchi_2021.tar.gz
# cleanup to free enough free space for the processing below
!rm yamchi_2021.tar.gz

## Define Processing Parameters

$\large\color{blue}{\text{Note: use prepared tiff and xml files in the DATADIR}}$

In [ ]:
# define supermaster image to align the images stack
SUPERMASTER  = '2021-02-13'
# define master image to build interferogram pairs 
MASTER       = '2021-04-26'
WORKDIR      = 'raw'
DATADIR      = 'yamchi_2021'
# coherence threshold
CORRLIMIT    = 0.40
DEFOMAX      = 0
# used to show SBAS baseline only, not for calculations
BASEDAYS     = 50
BASEMETERS   = 150

## Init SBAS

Search recursively for measurement (.tiff) and annotation (.xml) and orbit (.EOF) files in the DATA directory. It can be directory with full unzipped scenes (.SAFE) subdirectories or just a directory with the list of pairs of required .tiff and .xml files (maybe pre-filtered for orbit, polarization and subswath to save disk space). If orbit files and DEM are missed these will be downloaded automatically below.

In [ ]:
sbas = SBAS(DATADIR, basedir=WORKDIR).set_master(SUPERMASTER)

In [ ]:
sbas.to_dataframe()

### Download Sentinel-1 Orbits

The function below downloads orbit files.
Besides, for faster processing we can automatically use pre-downloaded orbit files in data directory.

In [ ]:
# use pre-downloaded orbit files in the unpacked data directory 
#sbas.download_orbits()

In [ ]:
#sbas.to_dataframe()

### Download SRTM DEM

The function below downloads SRTM1 or SRTM3 DEM and converts heights to ellipsoidal model using EGM96 grid.
Besides, for faster processing we can use pre-defined DEM file as explained above.

SRTM1 product is 30m resolution DEM and SRTM3 is 90m. SRTM1 is much bigger (~10 times) and is usable for small areas. Mainly 90m SRTM3 is the right choice. Use parameter resolution_meters (60 meters by default) to interpolate the DEM to required resolution for the future processing and output.

The DEM grid is NetCDF file.

In [ ]:
if platform.system() == 'Darwin':
    # this command allows to download DEM for a large area but it requires GMT 6.1+ and it does not work on Google Colab
    sbas.download_dem(backend='GMT')
else:
    # for old Linux Debian and Ubuntu use default backend to download SRTM3 DEM for large areas (4°x4° and more)
    #sbas.download_dem(product='SRTM3')
    # for old Linux Debian and Ubuntu use default backend to download SRTM1 DEM for small areas (less than 4°x4°)
    sbas.download_dem(product='SRTM1')

### Static Plots

In [ ]:
plt.figure(figsize=(12,4), dpi=300)
sbas.get_dem()[::4,::4].plot.imshow(cmap='gray', vmin=0)
plt.scatter(sbas.geoloc()['longitude'], sbas.geoloc()['latitude'], c=sbas.geoloc()['pixel'], cmap='jet')
plt.title('Sentinel1 Frame on DEM plus GCP', fontsize=18)
plt.show()

## Align a Pair of Images

In [ ]:
sbas.stack_parallel()

## DEM in Radar Coordinates

In [ ]:
sbas.topo_ra_parallel()

### Load Grids

In [ ]:
topo_ra = sbas.open_grid('topo_ra')

### Static Plot

In [ ]:
plt.figure(figsize=(12,4), dpi=300)
topo_ra[::4,::4].plot.imshow(cmap='gray', vmin=0)
plt.xlabel('Range', fontsize=16)
plt.ylabel('Azimuth', fontsize=16)
plt.title('Topography in Radar Coordinates', fontsize=18)
plt.show()

## Select Master Image Using SBAS Baseline

In [ ]:
baseline_pairs = sbas.baseline_pairs(days=BASEDAYS, meters=BASEMETERS)
baseline_pairs

### Static Plot

In [ ]:
plt.figure(figsize=(12,4), dpi=300)
ax = plt.gca()

lines = [[(row.ref_timeline,row.ref_baseline),(row.rep_timeline,row.rep_baseline)]
         for row in baseline_pairs.itertuples()]
lc = matplotlib.collections.LineCollection(lines, colors='#30a2da', linewidths=0.5)
ax.add_collection(lc)
ax.autoscale()
ax.margins(0.5)
bs1 = baseline_pairs[['ref_timeline','ref_baseline','ref_date']].values
bs2 = baseline_pairs[['rep_timeline','rep_baseline','rep_date']].values
df = pd.DataFrame(np.concatenate([bs1, bs2]), columns=['timeline','baseline','date']).drop_duplicates()
plt.scatter(x=df.timeline, y=df.baseline)
for x,y,label in df.values:
    plt.annotate(label, (x,y), textcoords="offset points", xytext=(35,3), ha='center',
                 c='red' if label == SUPERMASTER else 'black')
ax.set_xlabel('Timeline', fontsize=16)
ax.set_ylabel('Perpendicular Baseline, [m]', fontsize=16)
ax.set_title('SBAS Baseline', fontsize=18)
plt.grid()
plt.show()

## Interferogram

In [ ]:
# all the SBAS pairs
#pairs = baseline_pairs[['ref_date', 'rep_date']]
# or generate some kind of sequential pairs instead
pairs = list(sbas.to_dataframe().index)
# a. generate sequential master-secondary image pairs with different masters 
#pairs = [(p1, p2) for (p1, p2) in zip(pairs[:-1], pairs[1:])]
# b. generate master-secondary image pairs with the 1st image as single master 
#pairs = [(pairs[0], p) for p in pairs[1:]]
# c. generate master-secondary image pairs with the middle baseline image as single master 
pairs = [(MASTER, p) if MASTER<p else (p, MASTER) for p in pairs if p!=MASTER]
pairs

In [ ]:
# miss "func" argument when post-processing is not required
# define a postprocessing function for decimation, etc.
decimator = lambda dataarray: dataarray.coarsen({'y': 4, 'x': 4}, boundary='trim').median()

# default parameters: wavelength=200, psize=32, func=None (no postprocessing)
sbas.intf_parallel(pairs, func=decimator)

### Load Grids

In [ ]:
# crop grids for visualization
phasefilt = sbas.open_grids(pairs, 'phasefilt', geocode=True).sel(lat=slice(37.7, 38.4), lon=slice(47.9, 48.8))
corr = sbas.open_grids(pairs, 'corr', geocode=True).sel(lat=slice(37.7, 38.4), lon=slice(47.9, 48.8))

In [ ]:
# add fake master image to the stack
# self coherence is always equal to 1
before = corr[corr.ref<MASTER]
master = xr.ones_like(corr.isel(pair=0))
master['ref'] = master['rep'] = MASTER
master['pair'] = f'{MASTER} {MASTER}'
after = corr[corr.rep>MASTER]
mcorr = xr.concat([before, master, after], dim='pair')

### Static Plot

In [ ]:
fg = xr.where(mcorr>=CORRLIMIT, mcorr, np.nan).plot.imshow(
    col='pair',
    col_wrap=3, size=4, aspect=1.2,
    clim=(0, 1), cmap='gray'
)
fg.set_ticks(max_xticks=5, max_yticks=5, fontsize='medium')
fg.fig.suptitle('Coherence', y=1.02, fontsize=24)
plt.savefig(f'Coherence.jpg', dpi=150, quality=95)
plt.show()

In [ ]:
# fill low coherence areas by nearest values
phasefilt = xr.where(corr>=CORRLIMIT, phasefilt, np.nan)
phasefilt = xr.concat([sbas.nearest_grid(phasefilt[ida]) for ida in range(len(phasefilt))], dim='pair')
phasefilt = xr.where(np.isnan(corr), np.nan, phasefilt)

### Static Plot

In [ ]:
fg = phasefilt.plot.imshow(
    col='pair',
    col_wrap=3, size=4, aspect=1.2,
    vmin=-np.pi, vmax=np.pi, cmap='gist_rainbow_r'
)
fg.set_ticks(max_xticks=5, max_yticks=5, fontsize='medium')
fg.fig.suptitle('Phase, [rad]', y=1.02, fontsize=24)
plt.savefig(f'Phase, [rad].jpg', dpi=150, quality=95)
plt.show()

## Unwrapping

In [ ]:
# generate a custom snaphu config file and use it as argument "conf" value
# conf = self.PRM().snaphu_config(defomax=0)

# we can just miss "func" argument when post-processing is not required
# define a post-processing function to crop and interpolate low-coherence areas, etc.
#cleaner = lambda corr, unwrap: xr.where(corr>=CORRLIMIT, unwrap, np.nan)
cleaner = lambda corr, unwrap: sbas.nearest_grid(xr.where(corr>=CORRLIMIT, unwrap, np.nan))

# default parameters: threshold=0.1, conf=None, func=None (no postprocessing required)
sbas.unwrap_parallel(pairs, threshold=CORRLIMIT, func=cleaner)

## LOS Displacement

### Calculate Grids

In [ ]:
# crop grid for visualization
los_disp_mm = sbas.open_grids(pairs, 'unwrap', func=sbas.los_displacement_mm, geocode=True).sel(lat=slice(37.7, 38.4), lon=slice(47.9, 48.8))

In [ ]:
# fill low coherence areas by nearest values
los_disp_mm = xr.where(corr>=CORRLIMIT, los_disp_mm, np.nan)
los_disp_mm = xr.concat([sbas.nearest_grid(los_disp_mm[ida]) for ida in range(len(los_disp_mm))], dim='pair')
los_disp_mm = xr.where(np.isnan(corr), np.nan, los_disp_mm)

In [ ]:
# add fake master image to the stack
# master-master pair displacement is always equal to 0
before = los_disp_mm[los_disp_mm.ref<MASTER]
master = xr.zeros_like(los_disp_mm.isel(pair=0))
master['ref'] = master['rep'] = MASTER
master['pair'] = f'{MASTER} {MASTER}'
after = los_disp_mm[los_disp_mm.rep>MASTER]
mlos_disp_mm = xr.concat([-before, master, after], dim='pair')
#mlos_disp_mm

### Static Plots

In [ ]:
zmin, zmax = np.nanquantile(mlos_disp_mm, [0.01, 0.99])
fg = mlos_disp_mm.plot.imshow(
    col='pair',
    col_wrap=3, size=4, aspect=1.2,
    vmin=zmin, vmax=zmax, cmap='jet'
)
fg.set_ticks(max_xticks=5, max_yticks=5, fontsize='medium')
fg.fig.suptitle('LOS Displacement in Geographic Coordinates, [mm]', y=1.02, fontsize=24)
plt.savefig(f'LOS Displacement in Geographic Coordinates, [mm].jpg', dpi=150, quality=95)
plt.show()

## Check Coherence

In [ ]:
corr_dam = mcorr.sel(lon=(48.07808,48.08742), lat=(38.07058,38.06925), method='nearest')
corr_dam['pair'] = [da.rep.item() if da.pair.item()>MASTER else da.ref.item() for da in corr_dam]
#corr_dam

In [ ]:
plt.figure(figsize=(20,6))
corr_dam[:,0,0].plot(color='red', lw=1, ls='--', label='Top Left')
corr_dam[:,0,1].plot(color='black', lw=1, ls='--', label='Top Right')
corr_dam[:,1,0].plot(color='blue', lw=1, ls='--', label='Bottom Left')
corr_dam[:,1,1].plot(color='green', lw=1, ls='--', label='Bottom Right')
corr_dam.mean(['lat', 'lon']).plot(color='red', lw=2, label='Average')
plt.legend(loc='lower center', ncol=5, fontsize=24)
plt.grid()
plt.xlabel('Date', fontsize=24)
plt.ylabel('Coherence', fontsize=24)
plt.title('Yamchi Dam Coherence', fontsize=30)
plt.savefig(f'Yamchi Dam Coherence.jpg', dpi=150, quality=95)
plt.show()

In [ ]:
# Reference area
lat_min = mlos_disp_mm.lat.min()
lon_min = mlos_disp_mm.lon.min()
corr_ref = mcorr.sel(lat=slice(lat_min, lat_min+0.01), lon=slice(lon_min, lon_min+0.01))
corr_ref['pair'] = [da.rep.item() if da.pair.item()>MASTER else da.ref.item() for da in corr_ref]
corr_ref_min = corr_ref.min(['lat','lon'])
#corr_ref_min['pair'] = corr_ref_min['rep']
corr_ref_avg = corr_ref.mean(['lat','lon'])
#corr_ref_avg['pair'] = corr_ref_avg['rep']
corr_ref_max = corr_ref.max(['lat','lon'])
#corr_ref_max['pair'] = corr_ref_max['rep']

In [ ]:
plt.figure(figsize=(20,6))
corr_ref_min.plot(color='blue', lw=1, label='Minimum')
corr_ref_avg.plot(color='red', lw=1, label='Average')
corr_ref_max.plot(color='green', lw=1, label='Maximum')
plt.legend(loc='lower center', ncol=5, fontsize=24)
plt.grid()
plt.xlabel('Date', fontsize=24)
plt.ylabel('Coherence', fontsize=24)
plt.title('Yamchi Reference Area Coherence', fontsize=30)
plt.savefig(f'Yamchi Reference Area Coherence.jpg', dpi=150, quality=95)
plt.show()

## Yamchi Dam Displacement Analysis

In [ ]:
# Yamchi Dam points
los_disp_mm_dam = mlos_disp_mm.sel(lon=(48.07808,48.08742), lat=(38.07058,38.06925), method='nearest')
los_disp_mm_dam['pair'] = [da.rep.item() if da.pair.item()>MASTER else da.ref.item() for da in los_disp_mm_dam]
los_disp_mm_dam = xr.where(corr_dam>=CORRLIMIT, los_disp_mm_dam, np.nan)
#los_disp_mm_dam

In [ ]:
dates = [datetime.datetime.strptime(date, "%Y-%m-%d").timetuple().tm_yday for date in los_disp_mm_dam.pair.values]
values = los_disp_mm_dam.mean(['lat','lon']).values
trend_dam = np.polyfit(dates, values, 1)
print (trend_dam)
trendpoly = np.poly1d(trend_dam)
los_disp_mm_trend_dam = xr.zeros_like(los_disp_mm_dam.mean(['lat','lon']))
los_disp_mm_trend_dam.values = trendpoly(dates)
#los_disp_mm_trend_dam

In [ ]:
plt.figure(figsize=(20,6))
los_disp_mm_dam[:,0,0].plot(color='red', lw=1, ls='--', label='Top Left')
los_disp_mm_dam[:,0,1].plot(color='black', lw=1, ls='--', label='Top Right')
los_disp_mm_dam[:,1,0].plot(color='blue', lw=1, ls='--', label='Bottom Left')
los_disp_mm_dam[:,1,1].plot(color='green', lw=1, ls='--', label='Bottom Right')
los_disp_mm_trend_dam.plot(color='red', lw=2, label='Trend')
plt.legend(loc='lower center', ncol=5, fontsize=24)
plt.grid()
plt.xlabel('Date', fontsize=24)
plt.ylabel('LOS Displacement Velocity, [mm/12 days]', fontsize=24)
plt.title('Yamchi Dam LOS Displacement', fontsize=30)
plt.savefig(f'Yamchi Dam LOS Displacement.jpg', dpi=150, quality=95)
plt.show()

## Reference Area Displacement Analysis for Coherence Threshold 0.6

In [ ]:
# Reference area
lat_min = mlos_disp_mm.lat.min()
lon_min = mlos_disp_mm.lon.min()
los_disp_mm_ref = mlos_disp_mm.sel(lat=slice(lat_min, lat_min+0.01), lon=slice(lon_min, lon_min+0.01))
los_disp_mm_ref['pair'] = [da.rep.item() if da.pair.item()>MASTER else da.ref.item() for da in los_disp_mm_ref]
los_disp_mm_ref = xr.where(corr_ref>=CORRLIMIT, los_disp_mm_ref, np.nan).mean(['lat','lon'])
#los_disp_mm_ref

In [ ]:
dates = [datetime.datetime.strptime(date, "%Y-%m-%d").timetuple().tm_yday for date in los_disp_mm_ref.pair.values]
values = los_disp_mm_ref.values
trend_ref = np.polyfit(dates, values, 1)
print (trend_ref)
trendpoly = np.poly1d(trend_ref)
los_disp_mm_trend_ref = xr.zeros_like(los_disp_mm_ref)
los_disp_mm_trend_ref.values = trendpoly(dates)
#los_disp_mm_trend_ref

In [ ]:
plt.figure(figsize=(20,6))
los_disp_mm_ref.plot(color='black', lw=1, ls='--', label='Average')
los_disp_mm_trend_ref.plot(color='red', lw=2, label='Trend')
plt.legend(loc='lower center', ncol=5, fontsize=24)
plt.grid()
plt.xlabel('Date', fontsize=24)
plt.ylabel('LOS Displacement Velocity, [mm/12 days]', fontsize=24)
plt.title('Reference Area LOS Displacement', fontsize=30)
plt.savefig(f'Reference Area LOS Displacement.jpg', dpi=150, quality=95)
plt.show()

## Yamchi Dam vs Reference Area Displacement Analysis

In [ ]:
plt.figure(figsize=(20,6))
(los_disp_mm_dam[:,0,0]-los_disp_mm_ref).plot(color='red', lw=1, ls='--', label='Top Left')
(los_disp_mm_dam[:,0,1]-los_disp_mm_ref).plot(color='black', lw=1, ls='--', label='Top Right')
(los_disp_mm_dam[:,1,0]-los_disp_mm_ref).plot(color='blue', lw=1, ls='--', label='Bottom Left')
(los_disp_mm_dam[:,1,1]-los_disp_mm_ref).plot(color='green', lw=1, ls='--', label='Bottom Right')
(los_disp_mm_trend_dam-los_disp_mm_trend_ref).plot(color='red', lw=2, label='Trend')
plt.legend(loc='lower center', ncol=5, fontsize=24)
plt.grid()
plt.xlabel('Date', fontsize=24)
plt.ylabel('LOS Displacement Velocity, [mm/12 days]', fontsize=24)
plt.title('Yamchi Dam vs Reference Area LOS Displacement', fontsize=30)
plt.savefig(f'Yamchi Dam vs Reference Area LOS Displacement.jpg', dpi=150, quality=95)
plt.show()

## Build Output NetCDF 3D cubes

In [ ]:
#_los_disp_mm_ref = los_disp_mm_ref.copy()
#_los_disp_mm_ref['pair'] = los_disp_mm['pair']
_mlos_disp_mm = mlos_disp_mm.copy(deep=True)
_mlos_disp_mm['pair'] = [da.rep.item() if da.pair.item()>MASTER else da.ref.item() for da in _mlos_disp_mm]
los_disp_mm_deref = _mlos_disp_mm - los_disp_mm_ref
los_disp_mm_deref.to_netcdf('los_disp_mm_deref.nc')
#los_disp_mm_deref

In [ ]:
mcorr.to_netcdf('corr.nc')

### Export from Google Colab

In [ ]:
try:
    from google.colab import files
    files.download('los_disp_mm_deref.nc')
    files.download('corr.nc')
except:
    pass